In [18]:
import cv2      
import numpy as np 
import matplotlib.pyplot as plt  
import os
import math

## 计算在X坐标投影的非0像素个数 ##
def calXProjection(image,isSmooth, num):
    rows,cols = image.shape
    hist = np.zeros([cols],np.uint16)
    
    for i in range(cols):
        for j in range(rows):
            if image[j,i] > 200:
                hist[i] += 1
    
    if isSmooth :
        for i in range(cols):
            if hist[i] > 5:
                hist[i] = (hist[i]+hist[(i+1 + cols)%cols]+hist[(i+2+cols)%cols]+hist[(i-1 + cols)%cols]+hist[(i-2+cols)%cols])/5
            else :
                hist[i] = 0
        for i in range(cols):
            if hist[(i-2 + cols)%cols] == 0 and hist[(i+2 + cols)%cols] == 0:
                hist[i] = 0
                hist[(i-1 + cols)%cols] = 0
                hist[(i+1 + cols)%cols] = 0
    
    ### draw hist image for test  ###
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(hist)
    barH = 100
    histImg = np.zeros([barH+rows,cols], np.uint8)
    histImg[barH:barH+rows,:] = image
    hpt = int(0.9* barH)   
    for h in range(cols):    
        intensity = int(hist[h]*hpt/maxVal)    
        cv2.line(histImg,(h,barH), (h,barH-intensity), 255)
    histImgBig = cv2.resize(histImg,(2*cols, 2*(rows+barH)), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite("../../images/study/ROI/" + str(num) + "_3_hist.jpg", histImgBig, [int(cv2.IMWRITE_JPEG_QUALITY), 100]) 

#     cv2.imshow("Img_hist", histImgBig) 
#     cv2.waitKey(0)    
#     cv2.destroyAllWindows()
    return hist
    
    
def getROI(img, num):
    h, w = img.shape
    ##图像二值化##
    #Gaussian filtering
    img_seg = img[8:h,40:w-40]
    height, width = img_seg.shape
    
    img_blur = cv2.GaussianBlur(img_seg,(5,5),0)

    #自适应高斯二值化
    img_thre = cv2.adaptiveThreshold(img_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY_INV,11,3)
    #腐蚀
    kernel = np.ones((2,1),np.uint8)
    erosion = cv2.erode(img_thre, kernel, iterations = 3)
     #垂直方向膨胀
    kernel3 = np.ones((2,2),np.uint8)
    dilation = cv2.dilate(erosion,kernel3,iterations = 1)
    #计算像素x轴投影
    hist = calXProjection(dilation,True, num)
    
    start_pos = 0
    end_pos = width - 1
    for i in range(width - 4):
        if hist[i]+hist[i+1]+hist[i+2]+hist[i+3]+hist[i+4] > 25:
            start_pos = i+1
            break
    for i in range(width - 4):
        if hist[width - i -1]+hist[width - i -2]+hist[width - i -3]+hist[width - i -4]+hist[width - i -5] > 5:
            end_pos = width - i
            break
    
   
    cv2.imwrite("../../images/study/ROI/" + str(num) + "_2_threshold.jpg", dilation, [int(cv2.IMWRITE_JPEG_QUALITY), 100]) 
    img_ROI = dilation[1:height,start_pos:end_pos]
    return img_ROI


    
if __name__ == '__main__':
    for i in range(0,9):
        file =  "../../images/study/input/"+str(i)+".jpg"
        if not os.path.exists(file):
            continue
        img = cv2.imread(file, 0)  #直接读取成灰度图片
        # 输出原图
        cv2.imwrite("../../images/study/ROI/" + str(i) + "_1_ori.jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 100]) 
        # 输出ROI图像
        img_ROI = getROI(img, i)
        cv2.imwrite("../../images/study/ROI/" + str(i) + "_4_ROI.jpg", img_ROI, [int(cv2.IMWRITE_JPEG_QUALITY), 100]) 
